In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [2]:
from utils.imports import *

/home/twofyw/miniconda3/envs/tf/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


## Normalization

In [6]:
def open_greyscale(fn):
    """ Opens an image using OpenCV given the file path.

    Arguments:
        fn: the file path of the image

    Returns:
        The image in greyscale format as numpy array of floats normalized to range between 0.0 - 1.0
    """
    flags = cv2.IMREAD_GRAYSCALE
    if not os.path.exists(fn) and not str(fn).startswith("http"):
        raise OSError('No such file or directory: {}'.format(fn))
    elif os.path.isdir(fn) and not str(fn).startswith("http"):
        raise OSError('Is a directory: {}'.format(fn))
    else:
        #res = np.array(Image.open(fn), dtype=np.float32)/255
        #if len(res.shape)==2: res = np.repeat(res[...,None],3,2)
        #return res
        try:
            if str(fn).startswith("http"):
                req = urllib.urlopen(str(fn))
                image = np.asarray(bytearray(req.read()), dtype="uint8")
                im = cv2.imdecode(image, flags).astype(np.float32)/255
            else:
                im = cv2.imread(str(fn), flags).astype(np.float32)/255
            if im is None: raise OSError(f'File not recognized by opencv: {fn}')
            return im
        except Exception as e:
            raise OSError('Error handling image at: {}'.format(fn)) from e

In [7]:
subdir = '../data/SRAD2018_TRAIN_001/RAD_206482464212530/'

In [13]:
def calc_mean_std_subdir(subdir):
    """
    """
    ims = np.asarray([open_greyscale(o) for o in Path(subdir).glob('**/*.png')])
    mean, std = np.mean(ims), np.std(ims)
    return mean, std

In [15]:
par_dir = Path('../data/SRAD2018_TRAIN_001')
subdirs = [o for o in par_dir.iterdir() if o.is_dir()]

In [ ]:
subdir

In [22]:
%%time
means, stds = zip(*[calc_mean_std_subdir(o) for o in subdirs])

In [23]:
means, stds

((0.9820557,
  0.7564683,
  0.968596,
  0.9650422,
  0.98231244,
  0.7932429,
  0.9655625,
  0.97146374,
  0.9070648,
  0.7613075),
 (0.12885971,
  0.3993739,
  0.16909033,
  0.17655694,
  0.12847538,
  0.39483172,
  0.17846707,
  0.16026804,
  0.28292444,
  0.41495243))

In [29]:
mean , std = np.mean(stds), np.mean(means)

In [32]:
from data_reader import calc_mean_std_par_dir

In [38]:
mean, std = calc_mean_std_par_dir('../data/SRAD2018_TRAIN_001')

In [39]:
fn = 'stat.csv'
with open(fn, 'w') as f:
        np.array([mean, std]).tofile(f, ',')

In [40]:
!cat stat.csv

0.27102503180503845,0.8787143230438232

## SequenceGenerator

In [3]:
sess = tf.InteractiveSession()

In [4]:
shape = (61, 501, 501, 1)
def parser_train(serialized_example):
    features, sequence_features = tf.parse_single_sequence_example(
        serialized_example, context_features={
            'time_stamp': tf.FixedLenFeature([], tf.string),
        }, sequence_features={
            "raw_png": tf.FixedLenSequenceFeature([], dtype=tf.string)
        })
    
    x = tf.map_fn(tf.image.decode_png, sequence_features['raw_png'], dtype=tf.uint8,
                back_prop=False, swap_memory=False, infer_shape=False)
    x = tf.cast(x, tf.float32)
    x /= 255
    x.set_shape(shape)
    return x, features['time_stamp']

In [5]:
from utils.transforms import *

sz = 256
nt = 10
stats = np.fromfile('stat.csv', sep=',')

class Slice(Transform):
    """ Return a slice of the images
    
    Arguments:
    The same as the built-in function slice
    """
    def __init__(self, *args, **kwargs):
        self.slice = slice(*args, **kwargs)
        super().__init__(TfmType.NO)
        
    def do_transform(self, x, is_y):
        return x[self.slice]
    
aug_tfms = [Slice(nt)]
tfm, _ = tfms_from_stats(stats, sz, aug_tfms=aug_tfms, crop_type=CropType.NO)

SyntaxError: invalid syntax (transforms.py, line 14)

In [6]:
tfm

NameError: name 'tfm' is not defined

In [51]:
filenames = tf.placeholder(tf.string, name='tfrecords')
training_filenames = '../data/tfrecords/train_1.tfrecords'
dataset = tf.data.TFRecordDataset(filenames)
dataset = dataset.map(parser_train).map(tfm).repeat()
# dataset = dataset.prefetch()
iterator = dataset.make_initializable_iterator()
iterator.initializer.run({filenames: training_filenames})
x, time_stamp = iterator.get_next()

> /home/twofyw/pred-rain/yang/utils/transforms.py(12)scale_min()
     10     """
     11     shape = im.shape.as_list()
---> 12     if len(shape) == 4:
     13         shape = shape[1:]
     14     else if len(shape) != 3:

ipdb> q


BdbQuit: 

In [39]:
x.eval().shape

(10, 256, 256, 1)

## Prednet

In [28]:
from models.prednet import PredNet

In [38]:
from tensorflow.keras import backend as K

In [42]:
n_channels, im_height, im_width = (3, 128, 160)
input_shape = (n_channels, im_height, im_width) if K.image_data_format() == 'channels_first' else (im_height, im_width, n_channels)
stack_sizes = (n_channels, 48, 96, 192)
R_stack_sizes = stack_sizes
A_filt_sizes = (3, 3, 3)
Ahat_filt_sizes = (3, 3, 3, 3)
R_filt_sizes = (3, 3, 3, 3)
prednet = PredNet(stack_sizes, R_stack_sizes,
                  A_filt_sizes, Ahat_filt_sizes, R_filt_sizes,
                  output_mode='error', return_sequences=True, cell=tf.contrib.rnn.LSTMCell)

In [45]:
import tensorflow.keras as keras

In [53]:
nt = 10
inputs = keras.layers.Input(shape=(nt,) + input_shape)
km = keras.Model(prednet)

In [57]:
km.summary()

ValueError: This model has never been called, thus its weights have not yet been created, so no summary can be displayed. Build the model first (e.g. by calling it on some data).

## Keras

In [134]:
import tensorflow.keras as keras

In [136]:
rnn = keras.layers.RNN

In [138]:
rnn??